# Optimal control course project
# Solving the CartPole-v1 problem

by **Artem Petrov** and **Ivan Kudriakov**

Solving CartPole-v1 from openai.gym using methods of the Optimal Control Theory


# Step 1 
### (March 9, 2022)

## Mathematical model

The environment simulation of CartPole-v1 is performed according to the model, described here: https://coneural.org/florian/papers/05_cart_pole.pdf

([Proof from the code of CartPole-v1](https://github.com/openai/gym/blob/2dddaf722acccfd0412d745890c40dcd972586d5/gym/envs/classic_control/cartpole.py#L126))

Here, I will provide the brief explanation of the model

<img src="presentation_media/forces_model.png" alt="drawing" width="500"/>

<!-- Here $G_c$ and $G_p$ are the forces of gravity acting on cart and pole respectively.

Here $N_c$ - reaction force and $F_f$ - friction force between the cart and the rail -->

Original model considers the friction forces, however in CartPole-v1 there are no friction.

<!-- 
$\mu_c$ - friction coefficient between the cart and the rail

$\mu_p$ - friction coefficient between the cart and the pole

Side note: the initial model assumes Lubricated friction between the pole and the cart which is somewhat counterintuitive. -->

From the frictionless Newton equations the following  movement equations can be derived:

$$
\ddot{\theta} = \frac{g \sin \theta + \cos \theta \left( \frac{- F - m_p l \dot{\theta}^2\sin \theta}{m_c + m_p} \right)}
{l\left(\frac{4}{3} - \frac{m_p \cos ^2 \theta}{m_c + m_p}\right)}
\label{eq:theta} \tag{1}
$$

$$
\ddot{x} = \frac{F + m_p l\left(\dot{\theta}^2 \sin \theta - \ddot{\theta}\cos\theta\right)}
{m_c + m_p}
\label{eq:x} \tag{2}
$$

Where:

$m_c$ - the mass of the cart 

$m_p$ - the mass of the pole 

$l$ - half-length of the pole

$F = \pm F_0$ - horizontal force applied to the cart by the agent.

Interested reader who want to check out the whole inference of the equations above may want to read [the original source](https://coneural.org/florian/papers/05_cart_pole.pdf)


#### Now, let's pose the problem in terms of the optimal control theory.

The goal is to keep the pole upright for the longest time. 

Hence, one might suggests the usage of one the following simple functionals:

$$ 
\left[
\begin{array}[lll]
    .
    J_{\text{1}} = \int\limits_{0}^{\infty} \theta^2 dt \rightarrow \min \\
    J_{\text{2}} = \int\limits_{0}^{T} \theta^2 dt \rightarrow \min \\
    J_{\text{3}} = \int\limits_{0}^{T} |\theta| dt \rightarrow \min
\end{array}
\right .
$$ 


However, if we want to optimize for the highest value of the agent's reward function, the following functional must be used:

$$ J = T \rightarrow \max \tag{3} $$




Let $\mathbf{q} = (x, \dot{x}, \theta, \dot{\theta})^T$

$F = u F_0$, where $u \in \{-1, +1\}$ - control

Then, the dynamic constaraints can be written as follows:

$$\mathbf{\dot{q}} = 
\begin{bmatrix}
    \dot{x} \\
    \ddot{x} \\
    \dot{\theta} \\
    \ddot{\theta}
\end{bmatrix} = 
\begin{bmatrix}
    \dot{x} \\
    \frac{F + m_p l\left(\dot{\theta}^2 \sin \theta - \ddot{\theta}\cos\theta\right)}{m_c + m_p} \\
    \dot{\theta} \\
    \frac{g \sin \theta + \cos \theta \left( \frac{- F - m_p l \dot{\theta}^2\sin \theta}{m_c + m_p} \right)}
{l\left(\frac{4}{3} - \frac{m_p \cos ^2 \theta}{m_c + m_p}\right)}
\end{bmatrix} = 
\begin{bmatrix}
    \dot{x} \\
    \frac{uF_0 + m_p l\left(\dot{\theta}^2 \sin \theta - \frac{g \sin \theta + \cos \theta \left( \frac{- uF_0 - m_p l \dot{\theta}^2\sin \theta}{m_c + m_p} \right)}
{l\left(\frac{4}{3} - \frac{m_p \cos ^2 \theta}{m_c + m_p}\right)}\cos\theta\right)}{m_c + m_p} \\
    \dot{\theta} \\
    \frac{g \sin \theta + \cos \theta \left( \frac{- u F_0 - m_p l \dot{\theta}^2\sin \theta}{m_c + m_p} \right)}
{l\left(\frac{4}{3} - \frac{m_p \cos ^2 \theta}{m_c + m_p}\right)}
\end{bmatrix} = 
\mathbf{f}(\mathbf{q}, u)$$



Also, we must consider the fact, that in this particular implementation, the simulation will stop if one of the following conditions is reached:

- $|\theta| > 12^\circ$  (which contradicts the [frontpage](https://gym.openai.com/envs/CartPole-v1/), but is consistent with the [actual code](https://github.com/openai/gym/blob/2dddaf722acccfd0412d745890c40dcd972586d5/gym/envs/classic_control/cartpole.py#L90))

- $|x| > 2.4$

- $ t \le 500\tau$, where $\tau = 0.02$sec - is the length of one episode of simulation

These conditions can be formulated as the path constraints:

$$
\begin{cases}
    |q_1| = |x| \le 2.4\\
    |q_3| = |\theta| \le 12^\circ \\
    t \le 500\tau
\end{cases} \tag{4}
$$

However, because we maximize the time when our trajectory stays within the boundaries of 
$\begin{cases}
    |q_1| = |x| \le 2.4\\
    |q_3| = |\theta| \le 12^\circ
\end{cases}$, the last condition in (4) can be dropped, as the subtrajectory $\{\mathbf{q}(t)| t \in [0,500\tau]\}$ of a valid trajectory  $\{\mathbf{q}(t) | t \in [0,T]; T>500\tau\}$ will be the optimal solution for this problem/

#### Parameters of the environment

In order to get the model's parameters one can look at [the source code](https://github.com/openai/gym/blob/2dddaf722acccfd0412d745890c40dcd972586d5/gym/envs/classic_control/cartpole.py#L80) of the CartPole-v1 environment.

$g = 9.8$

$m_c = 1.0$ 

$m_p = 0.1$

$l = 0.5$ 

$F_0 = 10.0$


# Step 2
### (March 16 2022)

# LQR solution

## What is LQR (Linear–quadratic regulator)?

### Reference information (by [Ru Wiki LQR](https://ru.wikipedia.org/wiki/%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%BE-%D0%BA%D0%B2%D0%B0%D0%B4%D1%80%D0%B0%D1%82%D0%B8%D1%87%D0%BD%D1%8B%D0%B9_%D1%80%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%82%D0%BE%D1%80), [En Wiki LQR](https://en.wikipedia.org/wiki/Linear%E2%80%93quadratic_regulator))

If 
$$\dot{x} = A(t)x + B(t)u$$ 
and 
$$J = \int\limits_{0}^\infty \left( x^T Q(t) x + u^T R(t) u \right) dt$$ 
then optimal control is 
$$u = -R^{-1} B^T P x$$ 
when $P$ is solution of Ricatti equation: 
$$A^T P + P A - P B R^{-1} B^T P + Q = -\dot{P}$$

### At the last seminar we found:

$$\mathbf{\dot{q}} = 
\begin{bmatrix}
    \dot{x} \\
    \ddot{x} \\
    \dot{\theta} \\
    \ddot{\theta}
\end{bmatrix} = 
\begin{bmatrix}
    \dot{x} \\
    \frac{F + m_p l\left(\dot{\theta}^2 \sin \theta - \ddot{\theta}\cos\theta\right)}{m_c + m_p} \\
    \dot{\theta} \\
    \frac{g \sin \theta + \cos \theta \left( \frac{- F - m_p l \dot{\theta}^2\sin \theta}{m_c + m_p} \right)}
{l\left(\frac{4}{3} - \frac{m_p \cos ^2 \theta}{m_c + m_p}\right)}
\end{bmatrix} = 
\begin{bmatrix}
    \dot{x} \\
    \frac{uF_0 + m_p l\left(\dot{\theta}^2 \sin \theta - \frac{g \sin \theta + \cos \theta \left( \frac{- uF_0 - m_p l \dot{\theta}^2\sin \theta}{m_c + m_p} \right)}
{l\left(\frac{4}{3} - \frac{m_p \cos ^2 \theta}{m_c + m_p}\right)}\cos\theta\right)}{m_c + m_p} \\
    \dot{\theta} \\
    \frac{g \sin \theta + \cos \theta \left( \frac{- u F_0 - m_p l \dot{\theta}^2\sin \theta}{m_c + m_p} \right)}
{l\left(\frac{4}{3} - \frac{m_p \cos ^2 \theta}{m_c + m_p}\right)}
\end{bmatrix} = 
\mathbf{f}(\mathbf{q}, u)$$

### Linearization around the upper equilibrium of our system:
$$\mathbf{\dot{q}} = 
\begin{bmatrix}
    \dot{x} \\
    \ddot{x} \\
    \dot{\theta} \\
    \ddot{\theta}
\end{bmatrix} = 
\begin{bmatrix}
    0 & 1 & 0 & 0 \\
    0 & 0 & \frac{g}{l(\frac{4}{3} - \frac{m_p}{m_p+m_c})} & 0 \\
    0 & 0 & 0 & 1 \\
    0 & 0 & \frac{g}{l(\frac{4}{3} - \frac{m_p}{m_p+m_c})} & 0 
\end{bmatrix}q + 
\begin{bmatrix}
    0 \\
    \frac{1}{m_p+m_c} \\
    0 \\
    -\frac{1}{l(\frac{4}{3} - \frac{m_p}{m_p+m_c})}
\end{bmatrix}u $$


$$\mathbf{\dot{q}} = A\mathbf{q}+B\mathbf{u}$$

$$J = \int\limits_{0}^\infty \left( q^T Q q + u^T R u \right) dt$$

$$\mathbf{u} = K \mathbf{q} =  -R^{-1} B^T P \mathbf{q}$$
#### It means, in LQR control $\mathbf{u}$ is linear function of $\mathbf{q}$, but in inital problem it is discrete
But if you look at the formula, you can see that when constrained to the maximum force, the parameter $R$ conditions the proximity of the selection function $\mathbf{u}$ to $sign$.

#### Code below refers to [Optimal Control with OpenAI Gym](https://towardsdatascience.com/comparing-optimal-control-and-reinforcement-learning-using-the-cart-pole-swing-up-openai-gym-772636bc48f4)

In [1]:
import numpy as np

g = 9.8
lp = 0.5
mp = 0.1
mc = 1.0
mt = mp + mc # m_total


# state matrix
a = g/(lp*(4.0/3 - mp/mt))
A = np.array([[0, 1, 0, 0],
              [0, 0, a, 0],
              [0, 0, 0, 1],
              [0, 0, a, 0]])

# input matrix
b = -1/(lp*(4.0/3 - mp/mt))
B = np.array([[0], [1/mt], [0], [b]])

In [2]:
R = np.eye(1, dtype=int)          # choose R (weight for input)
print('R =', R)
Q = 5*np.eye(4, dtype=int)        # choose Q (weight for state)
print('Q =\n', Q)

# get riccati solver
from scipy import linalg

# solve ricatti equation
P = linalg.solve_continuous_are(A, B, Q, R)

# calculate optimal controller gain
K = np.dot(np.linalg.inv(R),
           np.dot(B.T, P))

R = [[1]]
Q =
 [[5 0 0 0]
 [0 5 0 0]
 [0 0 5 0]
 [0 0 0 5]]


In [3]:
def apply_state_controller(K, x):
    # feedback controller
    u = -np.dot(K, x)   # u = -Kx
    if u > 0:
        return 1, u     # if force_dem > 0 -> move cart right
    else:
        return 0, u     # if force_dem <= 0 -> move cart left

### In code below absolute value of force is changing on every step

In [4]:
import gym
# get environment
env = gym.make('CartPole-v0')
obs = env.reset()

for i in range(500):
    env.render()
    
    # get force direction (action) and force value (force)
    action, force = apply_state_controller(K, obs)
    
    # absolute value, since 'action' determines the sign, F_min = -10N, F_max = 10N
    abs_force = abs(float(np.clip(force, -10, 10)))
    
    # change magnitute of the applied force in CartPole
    env.env.force_mag = abs_force

    # apply action
    obs, reward, done, _ = env.step(action)
    if done:
        print(f'Terminated after {i+1} iterations.')
        break
env.close()

2022-03-14 21:58:58.564 Python[4408:186251] ApplePersistenceIgnoreState: Existing state will not be touched. New state will be written to /var/folders/1b/hpr4ztpn02947rpwmgc0gzph0000gn/T/com.apple.python3.savedState


Terminated after 200 iterations.


### Initially absolute value of force should be constant, so lets test LQR result with constant force

In [5]:
import gym
# get environment
env = gym.make('CartPole-v0')
obs = env.reset()

for i in range(500):
    env.render()
    
    # get force direction (action) and force value (force)
    action, force = apply_state_controller(K, obs)
    
    # absolute value, since 'action' determines the sign, F_min = -10N, F_max = 10N
    abs_force = abs(float(np.clip(force, -10, 10)))
    
    # change magnitute of the applied force in CartPole
    #env.env.force_mag = abs_force

    # apply action
    obs, reward, done, _ = env.step(action)
    if done:
        print(f'Terminated after {i+1} iterations.')
        break
env.close()

Terminated after 200 iterations.


# PID (proportional–integral–derivative controller)
#### Reference by Wiki ([EN](https://en.wikipedia.org/wiki/PID_controller), [RU](https://ru.wikipedia.org/wiki/%D0%9F%D0%98%D0%94-%D1%80%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%82%D0%BE%D1%80))

A proportional–integral–derivative controller (PID controller or three-term controller) is a control loop mechanism employing feedback that is widely used in industrial control systems and a variety of other applications requiring continuously modulated control. A PID controller continuously calculates an error value $e(t)$ as the difference between a desired setpoint (SP) and a measured process variable (PV) and applies a correction based on proportional, integral, and derivative terms (denoted P, I, and D respectively), hence the name.


$$u(t) = K_\text{p} e(t) + K_\text{i} \int_0^t e(\tau) \,\mathrm{d}\tau + K_\text{d} \frac{\mathrm{d}e(t)}{\mathrm{d}t}$$

#### Cart-Pole problem
Target value is $\theta=0$
$$u(t) = P\theta + I\int \theta + D \dot{\theta}$$

In our model we use sign of original control

In [6]:
import numpy as np
import gym

env = gym.make('CartPole-v1')

P, I, D = 1, 1, 1
tau = 0.02

for i_episode in range(5):
    state = env.reset()
    integral = 0
    derivative = 0
    prev_error = 0
    for t in range(500):
        env.render()
        integral += state[2] * tau
        derivative = state[3]
        error = state[2]
        res = I * integral + D * derivative + P * error
        if res>0:
            action = 1
        else:
            action = 0

        state, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps


In [7]:
import numpy as np
import gym

env = gym.make('CartPole-v1')

P, I, D = 1, 0, 1
tau = 0.02

for i_episode in range(5):
    state = env.reset()
    integral = 0
    derivative = 0
    prev_error = 0
    for t in range(500):
        env.render()
        integral += state[2] * tau
        derivative = state[3]
        error = state[2]
        res = I * integral + D * derivative + P * error
        if res>0:
            action = 1
        else:
            action = 0

        state, reward, done, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps
Episode finished after 500 timesteps


### Conclusion
Maybe this way can lead us to a good result, but it requires the selection of coefficients. These coefficients I tried are not bad, but there is often drift in the x-coordinate